# Example notebook for testing the task-launch endpoint

Requirements:
- obi-one service running locally: `make run-local`
- Access token: To be copied from platform and pasted below

In [ ]:
import httpx
import obi_one as obi
from entitysdk import Client, ProjectContext, models
from http import HTTPStatus

environment = "staging"
virtual_lab_id = obi.LAB_ID_STAGING_TEST
project_id = obi.PROJECT_ID_STAGING_TEST

token = "<COPY-TOKEN-HERE>"

# Get DB client
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
db_client = Client(environment=environment, project_context=project_context, token_manager=token)

# Get obi-one API client
OBI_ONE_API_URL = "http://127.0.0.1:8100"
headers = {
    "Authorization": f"Bearer {token}",
    "virtual-lab-id": virtual_lab_id,
    "project-id": project_id,
}
api_client = httpx.Client(base_url=OBI_ONE_API_URL, headers=headers)

## Example for circuit extraction

### Select a CircuitFromID and check its properties

In this example, we select a small circuit with 10 neurons as a parent circuit.

In [ ]:
circuit_id = "0182b55e-2f38-4e06-bbd0-b11e70449804"

circuit_from_id = obi.CircuitFromID(id_str=circuit_id)
circuit_entity = circuit_from_id.entity(db_client=db_client)

print()
print(circuit_entity.name)
print(circuit_entity.description)
print(f"#Neurons: {circuit_entity.number_neurons}, #Synapses: {circuit_entity.number_synapses}, #Connections: {circuit_entity.number_connections}")

### Set up circuit extraction to create a CircuitExtractionScanConfig registered in entitycore

In this example, we extract one sub-circuit containing 50% of the excitatory neurons from the parent circuit.

In [ ]:
# Create a CircuitExtractionScanConfig.initialize object
initialize = obi.CircuitExtractionScanConfig.Initialize(circuit=circuit_from_id)

# Create a CircuitExtractionScanConfig object with the initialize object
neuron_set=obi.PredefinedNeuronSet(node_set="Excitatory", sample_percentage=50)
info = obi.Info(campaign_name="EXC-Extraction", campaign_description="Extraction of percentages of EXC neurons")
scan_config = obi.CircuitExtractionScanConfig(initialize=initialize, neuron_set=neuron_set, info=info)

# Create the grid scan object
output_root = "../../../../obi-output/circuit_extraction_on_launch_system/grid_scan"
scan = obi.GridScanGenerationTask(
    form=scan_config,
    coordinate_directory_option="ZERO_INDEX",
    output_root=output_root
)

# Run the scan
# use_cache = False  # W/o cache, each circuit will be downloaded separatly for every scan coordinate (and deleted afterwards)
use_cache = True  # With cache, each circuit will only be downloaded once (and kept in the root folder of the scan)
scan.execute(db_client=db_client)

### Check generated camapaign and single coordinate config(s)

The generated campaign has been registered in entitycode under the IDs printed below.

In [ ]:
campaign_entity = scan.form.campaign
print(f"Campaign '{campaign_entity.name}' (ID {campaign_entity.id}): {campaign_entity.description}")

for cfg in scan.single_configs:
    print(f"  Coordinate {cfg.idx}: '{cfg.single_entity.name}' (ID {cfg.single_entity.id})")

### Take the first config and launch task on launch-system

When the task job was successfully submitted, the response will contain the execution activity ID. In case of more than one config in a campaign, each of them would need to be launched separately.

In [ ]:
# Select first config and check entity
single_config_entity = scan.single_configs[0].single_entity
single_config_type = single_config_entity.__class__.__name__
single_config_id = single_config_entity.id
print(f"Selected '{single_config_type}' with ID {single_config_id}")

In [ ]:
# Launch job
payload = {
    "task_type": "circuit_extraction",
    "config_id": str(single_config_id),
}
url = "/declared/task-launch"
response = api_client.post(url=url, json=payload, timeout=300.0)

In [ ]:
# Check status after job submission
if response.status_code == HTTPStatus.OK:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

### Check execution activity

The execution activity has information about the status, used config, executor job, and once done the extracted sub-circuit.

In [ ]:
execution_activity_id = data  # Response data is activity ID
execution_activity = db_client.get_entity(entity_type=models.CircuitExtractionExecution, entity_id=execution_activity_id)
job_id = execution_activity.execution_id
print(f"Activity '{str(execution_activity.type)}': ID {execution_activity_id}")
print(f"  Status: {execution_activity.status}")
for _used in execution_activity.used:
    print(f"  Used '{str(_used.type)}': ID {str(_used.id)}")
for _gen in execution_activity.generated:
    print(f"  Generated '{str(_gen.type)}': ID {str(_gen.id)}")
print(f"  Executed by '{str(execution_activity.executor)}': ID {job_id}")